In [1]:
# implementiramo branch and bound za problem ranca
# on sa slike koju je pokazao na casu (vidi snimak)

import queue

In [2]:
class Item:
    def __init__(self, weight, value):
        self.weight = weight
        self.value = value

In [3]:
class Node:
    def __init__(self, weight, value, level):
        self.weight = weight   # ukupna tezina predmeta od korena do ovog cvora
        self.value = value
        self.level = level
        
        self.bound = 0

In [4]:
# NOTE: morao sam da izadjem dok je pisao celu ovu fuknkciju, vidi na snimku
# objsanenje

# funkcija koja nam kaze gornju granicu, tj najbolje sto mozemo da dobijemo u tekucem cvoru
def bound(u, items, knapsackCapacity):
    # ovu procenu pravimo optimisticno i ovde kazemo da predmete mozemo da delimo na delove
    # tj da ne mora ceo predmet da stane u ranac. Ovo radimo jer samo trazimo gornju granicu, a ne
    # zapravo neko resenje (ako mozeom da delimo predmete)
    if u.weight >= knapsackCapacity:
        return 0
    totalWeight = u.weight
    totalValue = u.value
    level = u.level + 1 
    while level < len(items) and totalWeight < knapsackCapacity:
        totalValue += items[level].value
        totalWeight += items[level].weight
        level += 1
    if level != len(items):
        totalValue += items[level].value * (knapsackCapacity - totalWeight) / items[level].weight
    return totalValue

In [5]:
# branch and bound
def bnb(items, knapsackCapacity):
    # sortiramo po vrednosti po kilogramu
    items = sorted(items, key=lambda x : x.value/x.weight, reverse=True)
    Q = queue.Queue()
    # neki vestacki cvor koji ce biti iznad korena, da bismo uopste mogli da udjemo u petlju ispod
    node = Node(0, 0, -1)
    Q.put(node)
    maxValue = 0
    
    while not Q.empty():
        u = Q.get()
        if u.level == len(items) - 1:  # stigli do dna stabla
            continue;
            
        # cvor na kraju leve grane gde dodajemo item u ranac
        v_level = u.level + 1
        v = Node(u.weight + items[v_level].weight, 
                 u.value + items[v_level].value,
                 v_level)
        
        if v.weight <= knapsackCapacity and v.value > maxValue:
            maxValue = v.value
        
        v.bound = bound(v, items, knapsackCapacity)
        # ako u ovom cvoru moze da se dobije resenje bolje od poslednjeg, ima smisla da ga stavimo u red
        if v.bound > maxValue:
            Q.put(v)
        
        # cvor na kraju desne grane gde ne dodajemo item u ranac
        v = Node(u.weight, u.value, v_level)
        # i za desnu granu moramo istu proveru da imamo
        v.bound = bound(v, items, knapsackCapacity)
        if v.bound > maxValue:
            Q.put(v)
            
    return maxValue

In [7]:
# oni itemi sa slike za primer, treba da dobijemo 235
items = [Item(2, 40), Item(3.14, 50), Item(1.98, 100), Item(5, 95), Item(3, 30)]

bnb(items, knapsackCapacity=10)

235